In [2]:
# Импорт и базовый URL

import requests
import json
from pprint import pprint

BASE_URL = "http://127.0.0.1:8000"

# Получаем токен
USERNAME = "admin"
PASSWORD = "admin"
token_response = requests.post(
    f"{BASE_URL}/auth/token",
    data={"username": USERNAME, "password": PASSWORD}
)

if token_response.status_code != 200:
    raise Exception(f"Не удалось получить токен: {token_response.text}")

token = token_response.json()["access_token"]
headers = {"Authorization": f"Bearer {token}"}

print("✅ Токен получен:", token[:20], "...")  # выводим первые 20 символов

✅ Токен получен: eyJhbGciOiJIUzI1NiIs ...


In [3]:
# CREATE

lead_data = {
    "name": "Иван Иванов",
    "contact": "+70000000000",
    "log": json.dumps([{"role": "user", "message": "Привет"}]),
    "motive": "Боль в спине",
    "model": "Relaxio Compact",
    "needs": "Черный цвет"
}

response = requests.post(f"{BASE_URL}/lead/", json=lead_data, headers=headers)
print("CREATE:", response.status_code, response.json())

lead_id = response.json()["id"]  # сохраняем ID для следующих запросов

CREATE: 201 {'id': 2}


In [4]:
# READ ALL

response = requests.get(f'{BASE_URL}/lead', headers=headers)
print("GET ALL:", response.status_code)
pprint(response.json())

GET ALL: 200
[{'contact': '+70000000000',
  'id': 1,
  'log': '[{"role": "user", "message": '
         '"\\u041f\\u0440\\u0438\\u0432\\u0435\\u0442"}]',
  'model': 'Relaxio Compact',
  'motive': 'Боль в спине',
  'name': 'Иван Иванов',
  'needs': 'Черный цвет',
  'timestamp': '2025-09-28T17:28:47'},
 {'contact': '+70000000000',
  'id': 2,
  'log': '[{"role": "user", "message": '
         '"\\u041f\\u0440\\u0438\\u0432\\u0435\\u0442"}]',
  'model': 'Relaxio Compact',
  'motive': 'Боль в спине',
  'name': 'Иван Иванов',
  'needs': 'Черный цвет',
  'timestamp': '2025-09-28T17:29:49'}]


In [5]:
# READ ONE

response = requests.get(f"{BASE_URL}/lead/{lead_id}", headers=headers)
print("READ ONE:", response.status_code)
pprint(response.json())

READ ONE: 200
{'contact': '+70000000000',
 'id': 2,
 'log': '[{"role": "user", "message": '
        '"\\u041f\\u0440\\u0438\\u0432\\u0435\\u0442"}]',
 'model': 'Relaxio Compact',
 'motive': 'Боль в спине',
 'name': 'Иван Иванов',
 'needs': 'Черный цвет',
 'timestamp': '2025-09-28T17:29:49'}


In [6]:
update_data = {
    "motive": "Хроническая боль",
    "model": "Relaxio Deluxe"
}

response = requests.put(f"{BASE_URL}/lead/{lead_id}", json=update_data, headers=headers)
print("UPDATE:", response.status_code)

UPDATE: 200


In [7]:
# DELETE

response = requests.delete(f"{BASE_URL}/lead/{lead_id}", headers=headers)
print("DELETE:", response.status_code)  # 204 No Content

DELETE: 204


In [10]:
# Проверка загрузки текста документа
print(f"{BASE_URL}/base/document")
response = requests.get(f"{BASE_URL}/base/document", headers=headers)
if response.status_code == 200:
    data = response.json()
    print("✅ Успешно получили текст документа:")
    print("-" * 80)
    print(data["text"][:1000])  # выводим первые 1000 символов
else:
    print(f"Ошибка: {response.status_code}")
    print(response.text)

http://127.0.0.1:8000/base/document
✅ Успешно получили текст документа:
--------------------------------------------------------------------------------
# О компании  "Relaxio Comfort" 

 История компании

Компания "Relaxio Comfort" была основана в 2010 году группой энтузиастов, увлеченных идеей улучшения качества жизни через инновационные технологии. Основатели, вдохновленные современными исследованиями в области здоровья и комфорта, поставили перед собой цель сделать массаж доступным и эффективным в любом доме.

С момента создания компания активно сотрудничает с ведущими инженерами и специалистами в области физиотерапии, чтобы разрабатывать передовые модели массажных кресел. На сегодняшний день "Relaxio Comfort" является лидером в своей отрасли, предлагая продукцию премиум-класса, которая сочетает в себе функциональность, эстетичность и пользу для здоровья.

 Сфера деятельности

"Relaxio Comfort" специализируется на разработке, продаже и обслуживании массажных кресел высокого класса.

In [11]:
# Поисковый запрос
query_text = '''
    Чем занимается ваша компания?
'''
k_chunks = 5

# Формируем URL и JSON-тело
url = f"{BASE_URL}/base/chunks"
payload = {
    "query": query_text,
    "k": k_chunks
}

# Выполняем POST-запрос с телом
response = requests.post(url, json=payload, headers=headers)

# Проверяем результат
if response.status_code == 200:
    data = response.json()
    print("✅ Успешно получили релевантные чанки:")
    print("-" * 80)
    for i, chunk in enumerate(data["results"], 1):
        content_preview = chunk["content"][:200].replace("\n", " ")  # первые 200 символов
        print(f"{i}. Score: {chunk.get('score')} | Metadata: {chunk.get('metadata')}")
        print(f"   {content_preview}...")
        print("-" * 40)
else:
    print(f"Ошибка: {response.status_code}")
    print(response.text)

✅ Успешно получили релевантные чанки:
--------------------------------------------------------------------------------
1. Score: 0.4007781744003296 | Metadata: {'Header 1': 'Ключевые фразы для мотивации и общения', 'Header 2': 'Подчеркивание статуса и роскоши'}
   Подчеркивание статуса и роскоши   Фразы для клиентов, которым важно подчеркнуть свой статус:   "Для тех, кто выбирает лучшее из лучших."   "Инновации, стиль и забота в одном устройстве."   "Ваша зона ...
----------------------------------------
2. Score: 0.4075090289115906 | Metadata: {'Header 1': 'Стратегия общения с клиентами'}
   Стратегия общения с клиентами   1. Выявление потребностей   Правильно сформулированные вопросы помогают понять, что именно важно для клиента. Примеры:   "Как вы обычно снимаете усталость после напряже...
----------------------------------------
3. Score: 0.41079002618789673 | Metadata: {'Header 1': 'Контакты'}
   Контакты   Режим работы   Понедельник – Пятница: 9:00 – 20:00   Суббота – Воскресенье

In [12]:
# Проверим пересоздание базы
resp = requests.post(f"{BASE_URL}/base/rebuild", headers=headers)
print(resp.status_code, resp.json())

# Тест поиска чанков
query = {"query": "Условия доставки", "k": 3}
resp = requests.post(f"{BASE_URL}/base/chunks", json=query, headers=headers)
data = resp.json()

if resp.status_code == 200 and "results" in data:
    print("✅ Успешно получили релевантные чанки:")
    print("-" * 80)
    for i, chunk in enumerate(data["results"], 1):
        content_preview = chunk["content"][:200].replace("\n", " ")  # первые 200 символов
        print(f"{i}. Score: {chunk.get('score')} | Metadata: {chunk.get('metadata')}")
        print(f"   {content_preview}...")
        print("-" * 40)
else:
    print("❌ Ошибка:", resp.status_code, data)

200 {'status': 'ok', 'message': 'База успешно пересоздана'}
✅ Успешно получили релевантные чанки:
--------------------------------------------------------------------------------
1. Score: 0.3157559037208557 | Metadata: {'Header 1': 'Доставка'}
   Доставка   Бесплатная доставка по Москве и Санкт-Петербургу при заказе от 200 000 рублей.   Доставка по регионам России — от 3 до 7 рабочих дней.   Возможность экспресс-доставки в течение 1 дня по Мос...
----------------------------------------
2. Score: 0.3930799961090088 | Metadata: {'Header 1': 'Способы оплаты'}
   Способы оплаты   Мы предлагаем удобные и гибкие способы оплаты:   Банковские карты (Visa, MasterCard, Мир).   Онлайн-оплата через сайт.   Безналичный расчет для юридических лиц.   Рассрочка и кредит н...
----------------------------------------
3. Score: 0.415394127368927 | Metadata: {'Header 1': 'Дополнительные услуги'}
   Дополнительные услуги   Индивидуальная консультация в шоуруме или онлайн.   Подбор модели кресла в зависим

In [13]:
# Делаем запрос к /base/ask
ask_payload = {
    "query": "Условия доставки",
    "k": 5
}

ask_response = requests.post(f"{BASE_URL}/base/ask", json=ask_payload, headers=headers)

print(f"Status code: {ask_response.status_code}")

if ask_response.status_code == 200:
    answer = ask_response.json()["answer"]
    print("✅ Ответ GPT:")
    pprint(answer)
else:
    print("❌ Ошибка:")
    pprint(ask_response.json())
    

Status code: 200
✅ Ответ GPT:
('Если у вас есть вопросы по условиям доставки, я с радостью на них отвечу!\n'
 '\n'
 'Вот основные моменты, которые стоит учесть:\n'
 '\n'
 '1. **Бесплатная доставка**: бесплатная доставка доступна в Москве и '
 'Санкт-Петербурге при заказе от 200 000 рублей.\n'
 '\n'
 '2. **Доставка по регионам**: занимает от 3 до 7 рабочих дней.\n'
 '\n'
 '3. **Экспресс-доставка**: возможна доставка в течение 1 дня по Москве.\n'
 '\n'
 '4. **Сборка и настройка**: предлагается услуга сборки и настройки кресла на '
 'месте.\n'
 '\n'
 'Если у вас есть конкретные вопросы, уточните, и я помогу вам с ответами!')


In [2]:
client_id = 1

# Очистка истории перед началом диалога
resp = requests.post(
    f"{BASE_URL}/dialog/clear",
    json={"client_id": client_id},
    headers=headers,
)
if resp.status_code == 200:
    print(resp.json()["message"])
else:
    print(f"Ошибка очистки перед запуском: {resp.status_code} {resp.text}")

# Основной цикл общения
while True:
    user_input = input("Вы: ")
    if user_input.lower() in ("exit", "quit"):
        print("Выход из диалога")
        break
    elif user_input.lower() == "clear":
        resp = requests.post(
            f"{BASE_URL}/dialog/clear",
            json={"client_id": client_id},
            headers=headers,
        )
        if resp.status_code == 200:
            print(resp.json()["message"])
        else:
            print(f"Ошибка очистки: {resp.status_code} {resp.text}")
        continue

    # Отправляем запрос к /dialog/request
    print(f'Вы: {user_input}')
    resp = requests.post(
        f"{BASE_URL}/dialog/request",
        json={"user_input": user_input, "client_id": client_id},
        headers=headers
    )

    if resp.status_code == 200:
        data = resp.json()
        print("Ассистент:")
        pprint(data["response"])
        # при желании можно смотреть историю
        # print("История:", data["history"])
    else:
        print(f"Ошибка запроса: {resp.status_code} {resp.text}")

Диалог для клиента 1 очищен
Вы: Привет
Ассистент:
('Привет! Как я могу помочь вам сегодня? Если вас интересуют массажные кресла '
 'премиум-класса, их преимущества или что-то еще, дайте знать! Забота о вашем '
 'комфорте и здоровье — наша главная цель.')
Вы: Меня зовут Алексей. Какое кресло можешь предложить?
Ассистент:
('Привет, Алексей! Рад вас видеть. Для начала, скажите, какие у вас есть '
 'предпочтения или требования к массажному креслу? Например, вас интересуют '
 'функции массажа, размер кресла или может быть стиль и дизайн?\n'
 '\n'
 'Вот несколько моделей, которые могут вам подойти:\n'
 '\n'
 '1. **Relaxio Comfort Luxe**\n'
 '   - Отличается высочайшим качеством материалов и элегантным дизайном.\n'
 '   - Имеет множество режимов массажа, включая точечный и глубокий.\n'
 '   - Преимущества: идеален для расслабления после долгого рабочего дня, '
 'избавляет от напряжения в спине и шее.\n'
 '\n'
 '2. **Relaxio Comfort Classic**\n'
 '   - Более компактный вариант, отлично вписыва

In [3]:
# Тестируем саммаризацию
resp = requests.post(
    f"{BASE_URL}/dialog/summarize",
    json={"client_id": client_id},
    headers=headers,
)
print("Status:", resp.status_code)
pprint(resp.json())

Status: 200
{'client_id': 1,
 'summary': 'В диалоге Алексей интересуется покупкой массажного кресла и '
            'выражает желание выбрать самое дешевое. Собеседник предлагает '
            'модель Relaxio Comfort Classic, подчеркивая ее доступность, '
            'компактность и основные функции массажа. Также упоминается '
            'выгодное соотношение цены и качества, а также возможность '
            'рассрочки. Собеседник готов ответить на дополнительные вопросы и '
            'предоставить информацию о доставке.'}
